In [162]:
import xml.etree.ElementTree as ElementTree

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



In [163]:
!pip install xmltodict

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [164]:
import xml.etree.ElementTree as ElementTree

import xmltodict
import pprint
import json
import pandas as pd
import numpy as np



#tree = ElementTree.parse('sample.xml')
tree = ElementTree.parse('/Users/mina/Documents/NYU/BUILT/Matsim_Intermodal_comparison/raw/BUILT.30.experienced_plans.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)

#xmldict['person']


## Teresa: use xml object function to reshape data efficiently
-ref: https://stackoverflow.com/questions/1912434/how-do-i-parse-xml-in-python

In [165]:
#initiate activity related list object 
id_data_act = []
activity_data = []
start_time = []
for type_tag in root.findall('person'):
    for act in type_tag.findall('plan/activity'):
        id_data_act.append(type_tag.get('id')) #getting id
        activity_data.append(act.get('type'))  #getting work, home 
        start_time.append(act.get('start_time'))


In [166]:
for type_tag in root.findall('person'):
    if type_tag.get('id')  == '1007987':
        print(len(type_tag.findall('plan/leg')))
        for i ,mode in enumerate(type_tag.findall('plan/leg')):
            print(i)
            id_data_mode.append(type_tag.get('id')) 
            mode_data.append(mode.get('mode'))
            if i == len(type_tag.findall('plan/leg'))-1:
                print("hi")


2
0
1
hi


In [167]:
#initiate mode related list object 

id_data_mode = []
mode_data = []

for type_tag in root.findall('person'):
    if len(type_tag.findall('plan/leg')) == 0:
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append('na')
 
    for i ,mode in enumerate(type_tag.findall('plan/leg')):
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append(mode.get('mode'))
        if i == len(type_tag.findall('plan/leg'))-1:
            id_data_mode.append(type_tag.get('id')) 
            mode_data.append('N/A')

In [168]:
len(id_data_act)- len(id_data_mode) #exactly shows all ID has one less mode row (due to they arrive final destination)

0

In [169]:
len(mode_data), len(id_data_mode)

(26095, 26095)

In [170]:
actList =  list(zip(id_data_act, activity_data, start_time))
modeList = list(zip(id_data_mode, mode_data))

In [171]:
df_act = pd.DataFrame(actList, columns=['id','activity','start_time'])

In [172]:
#df_act.head(30)

In [173]:
df_mode = pd.DataFrame(modeList, columns=['id','mode'])
df_mode.head(50)


,id,mode
0,0,bike
1,0,bike
2,0,bike
3,0,bike
4,0,N/A
5,1,car
6,1,N/A
7,1007981,walk
8,1007981,transit_walk
9,1007981,N/A


In [184]:
df_mode.head()

,id,mode
0,0,bike
1,0,bike
2,0,bike
3,0,bike
4,0,N/A


In [175]:
merge = pd.merge(df_act, df_mode, how='left', left_on='id', right_on='id')
df_unique = merge.drop_duplicates(subset=['id','activity','start_time'],keep='first')

In [178]:
merge = pd.merge(df_act, df_mode, how='inner', left_on='id', right_on='id')

In [189]:
merge = pd.concat([df_act, df_mode],axis = 1)

In [190]:
merge.head(30)

,id,activity,start_time,id,mode
0,0,Work,None,0,bike
1,0,Work,10:21:16,0,bike
2,0,Secondary,13:36:27,0,bike
3,0,Work,13:52:13,0,bike
4,0,Home,20:12:56,0,N/A
5,1,Home,None,1,car
6,1,Work,06:24:23,1,N/A
7,1007981,Home,None,1007981,walk
8,1007981,Secondary,08:14:39,1007981,transit_walk
9,1007981,Work,12:37:14,1007981,N/A


In [161]:
df_mode[df_mode.id == '1007987']

,id,mode
23,1007987,transit_walk
24,1007987,walk
25,1007987,N/A


In [111]:
merge[merge.id =='1007987']

,id,activity,start_time,mode
50,1007987,Home,None,transit_walk
51,1007987,Home,None,transit_walk
52,1007987,Secondary,20:04:14,transit_walk
53,1007987,Secondary,20:04:14,transit_walk
54,1007987,Work,20:44:12,transit_walk
55,1007987,Work,20:44:12,transit_walk


In [17]:
xmldict['person'][1]['plan']['activity'][0]

{'type': 'Home',
 'link': '5671084_0',
 'end_time': '06:03:00',
 'activity': '\n\t\t\t'}

In [18]:
id_data = []
activity_data = []
mode_data = []
for row in xmldict['person']:
    #print(row['id'])
    id_data.append(row['id'])
    #print(row['plan']['activity'])
    activity_data.append(row['plan']['activity'])
    # Teresa: error handling
    try:
        mode_data.append(row['plan']['leg'])
    except KeyError:
        mode_data.append('NA')

        continue
    

In [19]:
# keep

id_data = []
activity_data = []
mode_data = []

# for row in xmldict['person']:
#     print(row['plan']['leg'])

for row in xmldict['person']:
    #print(row['id'])
    id_data.append(row['id'])
    #print(row['plan']['activity'])
    activity_data.append(row['plan']['activity'])
    # Teresa: error handling
    try:
        mode_data.append(row['plan']['leg'])
    except KeyError:
        mode_data.append('NA')

        continue
    
# data_tuples = list(zip(id_data, activity_data, mode_data))

#-----------------------
# print(id_data[2])
# print(activity_data[2][1]['type'])
#print(len(mode_data[2]))
#print(mode_data[2])


In [20]:
mode_data[0]

[{'mode': 'bike',
  'dep_time': '09:03:00',
  'trav_time': '01:18:16',
  'route': {'type': 'generic',
   'start_link': '226193928_0',
   'end_link': '420514148_1',
   'trav_time': '01:18:16',
   'distance': '19569.08484854245'}},
 {'mode': 'bike',
  'dep_time': '13:31:16',
  'trav_time': '00:05:11',
  'route': {'type': 'generic',
   'start_link': '420514148_1',
   'end_link': '458166896_5',
   'trav_time': '00:05:11',
   'distance': '1298.736213671756'}},
 {'mode': 'bike',
  'dep_time': '13:42:27',
  'trav_time': '00:09:46',
  'route': {'type': 'generic',
   'start_link': '458166896_5',
   'end_link': '588168464_0',
   'trav_time': '00:09:46',
   'distance': '2445.0770476881826'}},
 {'mode': 'bike',
  'dep_time': '18:52:13',
  'trav_time': '01:20:43',
  'route': {'type': 'generic',
   'start_link': '588168464_0',
   'end_link': '226193928_0',
   'trav_time': '01:20:43',
   'distance': '20180.827077682676'}}]

In [21]:
len(mode_data[1])

4

In [22]:
## keep finish separating 
##need to change print to add
mode = ['walk','bike','car','taxi']
act = ['Home','Secondary','Work','University','School']
new_list=[]
mode_mode_data = []
id_id_data=[]
activity_list = pd.DataFrame(columns=['type' , 'link', 'start_time' , 'end_time','activity'])



pt_count=0;
start = 0;
for i in range(0,10): #len(activity_data)
    start = 0;
    
#Teresa: added error handling
#     try :
#         key = mode_data[i][k]
#     except KeyError:
#         continue
#     #print(id_data[i])
    
    for j in range(0,len(activity_data[i])):
        pt_count=0

        if (activity_data[i][j]['type'] in act):
            #print(activity_data[i][j])
            id_id_data.append(id_data[i])
            #act_activity_data.append(activity_data[i][j]['type'])

            ###add activity to list
            new_list.append(activity_data[i][j])
            #print(new_list)
            
            ### append list to DataFrame
            activity_list= activity_list.append(new_list, ignore_index=True)
            new_list.clear()


#Teresa: the k loop should not be an inner loop of j loop (it doesn't use j index)
#Seems the type 'car' has different format? you can skip it by using 'try...except...continue' at the outer i loop
        for k in range(start,len(mode_data[i])):

            try :
                key = mode_data[i][k]
            except KeyError:
                print(mode_data[i])
                mode_mode_data.append('skip')
                continue      
            if mode_data[i][k]['mode'] in mode:
                mode_mode_data.append(mode_data[i][k]['mode'])
                #print(i,j,k,mode_data[i][k])
                start=k+1;
                break;

            elif mode_data[i][k]['mode']=='pt': 
                if(pt_count ==0):
                    mode_mode_data.append(mode_data[i][k]['mode'])
                    #print(i,j,k,mode_data[i][k])
                    pt_count=1 
            elif mode_data[i][k]['mode']=='egress_walk' and pt_count==1:
                start = k+1;
                break;

            else:
                continue;






    mode_mode_data.append('')
    
    
print("data mining done")


{'mode': 'car', 'dep_time': '06:03:00', 'trav_time': '00:21:23', 'route': {'type': 'links', 'start_link': '5671084_0', 'end_link': '8948920_0', 'trav_time': '00:21:23', 'distance': '29089.532874252345', 'route': '5671084_0 195743195_0 195743195_1 195743195_2 226193905_0_r 46165816_0_r 438363728_0_r 413052676_0 195743264_0 438900013_0 438900013_1 420809440_0 195743228_0 421832645_0 222299274_0 195743229_0 195743226_0 516664210_0 516664206_0 413050334_0 413050334_1 413050334_2 413050334_3 496982371_0 588157330_0 438370108_0 420809441_0 420809420_0 420809420_1 420809420_2 420809420_3 195743557_0 195743558_0 195743558_1 195743558_2 195743558_3 420522049_0 420522049_1 420522049_2 420522049_3 473884145_0 505776502_0 22903923_0 34625442_0 194923802_0 194923802_1 194923802_2 194923801_0 464847780_0 34625463_0 194923804_0 22903913_0 194923803_0 118714476_0 118714476_1 118714476_2 118714476_3 118714476_4 194926850_0 421853810_0 421853810_1 421853810_2 421853809_0 68325061_0 68325061_1 421853647_

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [23]:
#id and mode data tuple
data_tuples = list(zip(id_id_data, mode_mode_data))

data_tuples

#keep -make a DataFrame

df = pd.DataFrame(data_tuples, columns=['id','mode'])



#df
print("dataframe succeed")

dataframe succeed


In [24]:
#merge index list and activity list

df_merged = pd.concat([df, activity_list], axis=1)
df_merged


,id,mode,type,link,start_time,end_time,activity
0,0,bike,Work,226193928_0,NaN,09:03:00,\n\t\t\t
1,0,bike,Work,420514148_1,10:21:16,13:31:16,\n\t\t\t
2,0,bike,Secondary,458166896_5,13:36:27,13:42:27,\n\t\t\t
3,0,bike,Work,588168464_0,13:52:13,18:52:13,\n\t\t\t
4,0,,Home,226193928_0,20:12:56,NaN,\n\t\t\t
5,1,skip,Home,5671084_0,NaN,06:03:00,\n\t\t\t
6,1,skip,Work,8948920_0,06:24:23,11:53:23,\n\t\t\t
7,1007981,skip,Home,421854005_0,NaN,07:52:00,\n\t\t\t
8,1007981,skip,Secondary,46694764_3,08:14:39,08:19:39,\n\t\t\t
9,1007981,skip,Work,438384693_1,12:37:14,22:54:14,\n\t\t\t


In [25]:
df_merged['start_time']=pd.to_datetime(df_merged['start_time'])
df_merged['end_time']=pd.to_datetime(df_merged['end_time'])


#df_merged['end_time']=pd.to_datetime(df_merged['end_time'],format= '%H:%M:%S' ).dt.time





In [26]:
# check travel time
trave_time = pd.DataFrame(columns={'id','travel_time'})

for i in range(0,len(df_merged)-1):
    if(df_merged.iloc[i]['id']==df_merged.iloc[i+1]['id']):
        diff =pd.Timedelta(df_merged.iloc[i]['end_time']  - df_merged.iloc[i+1]['start_time'] ).seconds / 3600
    else:
        diff = np.nan
    
    trave_time=trave_time.append([{'id': df_merged.iloc[i]['id'], 'travel_time':diff}], ignore_index=True)




In [27]:
#merge 

df_final =  pd.concat([df_merged, trave_time['travel_time']], axis=1)
df_final

,id,mode,type,link,start_time,end_time,activity,travel_time
0,0,bike,Work,226193928_0,NaT,2019-05-22 09:03:00,\n\t\t\t,22.695556
1,0,bike,Work,420514148_1,2019-05-22 10:21:16,2019-05-22 13:31:16,\n\t\t\t,23.913611
2,0,bike,Secondary,458166896_5,2019-05-22 13:36:27,2019-05-22 13:42:27,\n\t\t\t,23.837222
3,0,bike,Work,588168464_0,2019-05-22 13:52:13,2019-05-22 18:52:13,\n\t\t\t,22.654722
4,0,,Home,226193928_0,2019-05-22 20:12:56,NaT,\n\t\t\t,NaN
5,1,skip,Home,5671084_0,NaT,2019-05-22 06:03:00,\n\t\t\t,23.643611
6,1,skip,Work,8948920_0,2019-05-22 06:24:23,2019-05-22 11:53:23,\n\t\t\t,NaN
7,1007981,skip,Home,421854005_0,NaT,2019-05-22 07:52:00,\n\t\t\t,23.622500
8,1007981,skip,Secondary,46694764_3,2019-05-22 08:14:39,2019-05-22 08:19:39,\n\t\t\t,19.706944
9,1007981,skip,Work,438384693_1,2019-05-22 12:37:14,2019-05-22 22:54:14,\n\t\t\t,NaN


In [28]:
#group by id for travel_time

####### add mode on the final group by
total_travel_time = df_final.groupby(['id']).sum()
total_travel_time

#len(travel_time_comparison)

,travel_time
id,
0,93.101111
1,23.643611
1007981,43.329444
1007982,13.086667
1007983,42.995278
1007984,42.589167
1007985,13.489722
1007986,34.708889
1007987,34.830000


In [29]:
df_final.to_excel("10th_mode_final.xlsx")



ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
total_travel_time.to_excel("10th_totalTime_final.xlsx")
#df_merged.to_csv("sample2.csv",sep='\t', encoding='utf-8')
print("all done")